In [ ]:
import pandas as pd #import pandas to access Data Frame
import numpy as np #import Numpy for Data Frame manipulation
from bs4 import BeautifulSoup as bs #import beatiful soup for web scrapping
import requests #import requests to fetch records

Orig_fortune = open("fortune1000.txt","r") # original fortune 1000 text file

#Reading text file into list
newdata = Orig_fortune.readlines()

#Reading records into list
rawdatalist = [line.strip() for line in newdata]

#converting list into array
rawarray = np.array(rawdatalist)

#creating dataframe and reshaping it
df2 = pd.DataFrame(rawarray.reshape(1000,11), columns = ['Rank' ,'Name','Revenue($m)','%change in Revenue','Profit($m)','% Change in Profits', 'Assets($m)','Market_Value($m)','Change in Rank (1000)', 'Employees', 'Change in Rank (500 Only)'] )

#dropping unnecessary data
df2.drop(['%change in Revenue','% Change in Profits','Change in Rank (1000)','Change in Rank (500 Only)'],axis=1,inplace =True)

#replacing special characters with blank
df2['Revenue($m)'] = df2['Revenue($m)'].str.replace('$','')
df2['Revenue($m)'] = df2['Revenue($m)'].str.replace(',','')
df2['Profit($m)'] = df2['Profit($m)'].str.replace('$','')
df2['Profit($m)'] = df2['Profit($m)'].str.replace(',','')
df2['Assets($m)'] = df2['Assets($m)'].str.replace('$','')
df2['Assets($m)'] = df2['Assets($m)'].str.replace(',','')
df2['Market_Value($m)'] = df2['Market_Value($m)'].str.replace('$','')
df2['Market_Value($m)'] = df2['Market_Value($m)'].str.replace(',','')
df2['Employees'] = df2['Employees'].str.replace(',','')

#saving the file to csv
df2.to_csv('Fortune1000_TransformData.csv')


# Srcapping data
#Reading Csv file into dataframe
df = pd.read_csv('Fortune1000_TransformData.csv')
comp_url = '.websiteoutlook.com'

#assigning URL from dataframe to a list variable
url_list = df.URL

#Appending the url list for analysis
update_list = url_list + comp_url

#list variables to store data
Alexa_rank = []
Backlinks = []
Page_authority = []
Domain_authority = []
Moz_Rank = []
Page_views = []
worth = []
Page_size = []
code_to_text_ratio = []
Sem_rank = []
Keyword = []
organic_traffic = []
cost = []
adv_keyword = []
adv_traffic= []
adv_budget = []

#Fetching Records
for url in update_list[:20]:   #for 20 records
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    Alexa_rank.append(soup.find('span',{"class":"label label-primary"}).get_text().strip())
    Backlinks.append(soup.find('span',{"class":"label label-default"}).get_text().strip())
    Page_authority.append(soup.find('span',{"class":"label label-info"}).get_text().strip())
    Domain_authority.append(soup.find('span',{"class":"label label-info"}).get_text().strip())
    Moz_Rank.append(soup.find('span',{"class":"label label-info"}).get_text().strip())
    Page_views.append(soup.find('span',{"class":"label label-warning"}).get_text().strip())
    worth.append(soup.find('span',{"class":"label label-danger"}).get_text().strip())
    Page_size = soup.find("dt",text="Page Size").findNext("dd").string.strip() #To find the next DD tag and fetch specific data. Most efficient way
    code_to_text_ratio = soup.find("dt",text="Code to Text Ratio").findNext("dd").string.strip()
    table_data = soup.find("table",{"class":"table table-bordered table-condensed"}) #Fetching table records
    str_words = table_data.find_all("td")
    Sem_rank.append((str_words[1]).get_text())
    Keyword.append((str_words[4]).get_text())
    organic_traffic.append((str_words[7]).get_text())
    cost.append((str_words[10]).get_text())
    adv_keyword.append((str_words[13]).get_text())
    adv_traffic.append((str_words[16]).get_text())
    adv_budget.append((str_words[19]).get_text())
    
#storing list variables into dataframe    
df1 = pd.DataFrame(list(zip(Alexa_rank,Backlinks,Page_authority,Domain_authority,Moz_rank,Page_views,worth,Page_size,code_to_text_ratio,Sem_rank,Keyword,organic_traffic,cost,adv_keyword,adv_traffic,adv_budget)),columns = ["Alexa Rank","Backlinks","Page Authority","Domain Authority","Moz Rank","Page Views","Worth","Page Size","Code to text Ratio%","Sem Rank","Keyword","Organic Traffic","Cost","Adwords Keyword","Adwords Traffic","Adwords Budget"])    
    
#saving dataframe into csv file    
df1.to_csv('Scrapping_data.csv') 

# ****IMPORTANT*****

#Last Point in pdf ask us to combine the two dataframe which I suppose is logically incorrect because the data inside the second dataframe belongs to the first record in first dataframe which implies it to be a dataframe multidimentional array and such thing can't be stored in a csv file.